<a href="https://colab.research.google.com/github/ABSplus/furrydiffusion-coalab/blob/main/FurryDiffusion's_(now_w_XFORMERS!)_StableDiffusionUI_Voldemort.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> Adapted from: [TheLastBen's Optimized Colab](https://github.com/TheLastBen/fast-stable-diffusion) which is a derivative of [MattieuHPTR's Optimizations](https://github.com/huggingface/diffusers/pull/532)


Join us on the [Furry Diffusion Discord](https://discord.gg/pxpEYkmphQ)

# Download the requirements

In [ ]:
%%capture
%mkdir /content/sd
%cd /content/sd
!git clone https://github.com/CompVis/stable-diffusion
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui
%cd /content/sd/stable-diffusion-webui/
!pip install huggingface
!pip install torch

In [ ]:
import time
%cd /content/sd/stable-diffusion-webui
!mkdir repositories
%cd /content/sd
!git clone https://github.com/CompVis/taming-transformers /content/sd/taming-transformers
!git clone https://github.com/openai/CLIP /content/sd/CLIP
!git clone https://github.com/salesforce/BLIP /content/sd/BLIP
!git clone https://github.com/sczhou/CodeFormer /content/sd/CodeFormer
!git clone https://github.com/xinntao/Real-ESRGAN /content/sd/realESRGAN
!git clone https://github.com/Hafiidz/latent-diffusion /content/sd/latent-diffusion
!git clone https://github.com/crowsonkb/k-diffusion.git /content/sd/k-diffusion
!git clone https://github.com/TencentARC/GFPGAN /content/sd/stable-diffusion-webui/repositories/GFPGAN
%cd /content/
!wget https://github.com/TheLastBen/fast-stable-diffusion/raw/main/Dependencies/Dependencies_AUT.1
!wget https://github.com/TheLastBen/fast-stable-diffusion/raw/main/Dependencies/Dependencies_AUT.2
!mv Dependencies_AUT.1 Dependencies_AUT.7z.001
!mv Dependencies_AUT.2 Dependencies_AUT.7z.002
!7z x Dependencies_AUT.7z.001
time.sleep(3)
!cp -r /content/usr/local/lib/python3.7/dist-packages /usr/local/lib/python3.7/
!rm -r /content/usr
!rm Dependencies_AUT.7z.001
!rm Dependencies_AUT.7z.002
!cp -r /content/sd/stable-diffusion/ldm /content/sd/stable-diffusion-webui/
time.sleep(1)
%cd /content/sd/stable-diffusion-webui/ldm/modules
!wget -O attention.py https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/precompiled/attention.py
%cd /content/
!git clone https://github.com/DominikDoom/a1111-sd-webui-tagcomplete /content/autocomplete
!sed -i 's/danbooru.csv/e621.csv/g' /content/autocomplete/tags/config.json
!cp -r /content/autocomplete/tagAutocomplete.js /content/sd/stable-diffusion-webui/javascript/tagAutocomlete.js
!cp -r /content/autocomplete/tags /content/sd/stable-diffusion
time.sleep(1)
!rm -r /content/autocomplete
print('Completed.')

In [ ]:
from IPython.display import clear_output
import time
from IPython.display import HTML
from subprocess import getoutput
import os

def gpu_t4() -> None:
  %pip install -q https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/T4/xformers-0.0.13.dev0-py3-none-any.whl
  return

def gpu_p100() -> None:
  %pip install -q https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/V100/xformers-0.0.13.dev0-py3-none-any.whl
  return

def gpu_v100() -> None:
  %pip install -q https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/P100/xformers-0.0.13.dev0-py3-none-any.whl
  return

def gpu_a100() -> None:
  %pip install -q https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/A100/xformers-0.0.13.dev0-py3-none-any.whl
  return

hashmap_gpus = {
    "T4": gpu_t4,
    "P100": gpu_p100,
    "V100": gpu_v100,
    "A100": gpu_a100
}

gpu_func = None

s = getoutput('nvidia-smi')
if 'T4' in s:
  gpu = 'T4'
elif 'P100' in s:
  gpu = 'P100'
elif 'V100' in s:
  gpu = 'V100'
elif 'A100' in s:
  gpu = 'A100'

gpu_func = hashmap_gpus[gpu]

if not gpu_func:
  print('it seems that your GPU is not supported at the moment')
  time.sleep(5)
else:
  gpu_func()

print('Completed.')

# Download a model

### **Recommended:** Directly download a model

In [ ]:
hashmap_models = {
    'fe4':'furry-ckpt/furry_epoch4',
    'ye18':'yiffy-ckpt/yiffy-e18',
    'z3dv1':'furry-ckpt/Zack3D_Kinky-v1',
    'ponyv1':'furry-ckpt/pony_sfw_80k_safe_and_suggestive_500rating_plus-pruned',
    'cookie-pony-nsfw':'furry-ckpt/cookie_sd_pony_nsfw_run_a9_datasetv3_28k_imgs',
    'krystal':'furry-ckpt/dreambooth_krystal_thatbluefoxfromstarfoxkrystal-anthro'
}

selected_model = 'fe4' # @param ["fe4", "ye18", "z3dv1", "ponyv1", "cookie-pony-nsfw", "krystal"]

!wget https://sexy.canine.wf/file/{hashmap_models[selected_model]}.ckpt -O /content/sd/stable-diffusion-webui/models/Stable-diffusion/model.ckpt

print('Completed.')

### **OR** Import a custom model from Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

file_id = "model.ckpt" #@param {type:"string"}

!cp /content/drive/MyDrive/{file_id} /content/sd/stable-diffusion-webui/models/Stable-diffusion/model.ckpt

print('Completed.')

# The Home Stretch! **Almost done!**

In [ ]:
#@markdown # When the AI is ready to be used, it will say: <br />
#@markdown Running on local URL: **htt\*://xxxxxxxxxxxxxxxx.loca.lt:443** <br />
#@markdown Open that link. It's a website. If it's the first time you've tried colab, it will give you a little message. You just have to press "Click to continue" <br />
#@markdown This step will not 'complete' like the last steps, it's completely normal and intended behaviour. <br />
from IPython.utils import capture
from subprocess import getoutput
import os
import time

with capture.capture_output() as cap: 
  if not os.path.exists('/tools/node/bin/lt'):
    !npm install -g localtunnel

with capture.capture_output() as cap: 
  %cd /content/sd/stable-diffusion-webui/
  time.sleep(1)
  !wget -O webui.py https://raw.githubusercontent.com/AUTOMATIC1111/stable-diffusion-webui/master/webui.py
  !sed -i 's@gpu_call).*@gpu_call) \n        demo.queue(concurrency_count=111500)@' /content/sd/stable-diffusion-webui/webui.py
Use_Gradio_Server = False #@param {type:"boolean"}
#@markdown  - Only if you have trouble connecting to the local server
with capture.capture_output() as cap: 
  %cd /content

share=''
if Use_Gradio_Server:
  share='--share'
  !sed -i '1037s@.*@            self.server_name = server_name@' /usr/local/lib/python3.7/dist-packages/gradio/blocks.py
  !sed -i '1039s@.*@            self.server_port = server_port@' /usr/local/lib/python3.7/dist-packages/gradio/blocks.py  
  !sed -i '1043s@.*@            self.protocol = "https" if self.local_url.startswith("https") else "http"@' /usr/local/lib/python3.7/dist-packages/gradio/blocks.py  
else:
  !nohup lt --port 7860 > srv.txt 2>&1 &
  time.sleep(2)
  !grep -o 'https[^ ]*' /content/srv.txt >srvr.txt
  time.sleep(2)
  srv= getoutput('cat /content/srvr.txt')

  !sed -i '1037s@.*@            self.server_name = "{srv[8:]}"@' /usr/local/lib/python3.7/dist-packages/gradio/blocks.py
  !sed -i '1039s@.*@            self.server_port = 443@' /usr/local/lib/python3.7/dist-packages/gradio/blocks.py
  !sed -i '1043s@.*@            self.protocol = "https"@' /usr/local/lib/python3.7/dist-packages/gradio/blocks.py  
          
  !sed -i '13s@.*@    "PUBLIC_SHARE_TRUE": "[32mConnected",@' /usr/local/lib/python3.7/dist-packages/gradio/strings.py
  
  !rm /content/srv.txt
  !rm /content/srvr.txt

with capture.capture_output() as cap: 
  %cd /content/sd/stable-diffusion

!python /content/sd/stable-diffusion-webui/webui.py $share --disable-safe-unpickle

# Download outputs from Colab

Run the code, navigate to the left hotbar, look for the folder icon and right click the zip file and hit download!

In [ ]:
from google.colab import files

%cd /content/sd/stable-diffusion/

!zip -r /content/images.zip outputs log/images

files.download('/content/images.zip')